<a href="https://colab.research.google.com/github/javier-jaime/Route-Recommender/blob/main/Distance_Matrix.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
import pandas as pd
# import googlemaps


from google.colab import drive
from google.colab import files

In [5]:
# Mount Google Drive

drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [207]:
#Read Excel file into dataframe
cities = pd.read_csv('drive/MyDrive/Colab_Data/Western_cities.csv', index_col=0)
print(len(cities))
cities

30


,Destination Province,Destination City,Latitude,Longitude
0,AB,ACHESON,53.545914,-113.771576
1,AB,AIRDRIE,51.285970,-114.010620
2,AB,BALZAC,51.215611,-114.004558
3,AB,CALGARY,51.046095,-114.065465
4,AB,EDMONTON,53.535411,-113.507996
5,AB,GRANDE PRAIRIE,55.171080,-118.794987
6,AB,LETHBRIDGE,49.694285,-112.851562
7,AB,MEDICINE HAT,50.043030,-110.679016
8,AB,OKOTOKS,50.725369,-113.975082
9,AB,RED DEER COUNTY,52.104022,-113.767473


 From OSRM (Open Source Routing Machine)
 https://github.com/Project-OSRM/osrm-backend/wiki/Server-api

In [213]:
route_uri = 'http://router.project-osrm.org/route/v1/driving/'
loc_pair = '13.388860,52.517037;13.397634,52.529407'
file = pd.read_json( route_uri + loc_pair,lines=True)
distance = file['routes'][0][0]['distance']
duration = file['routes'][0][0]['duration']

In [214]:
# Returns a 3x3 duration matrix and load into a dataframe (from json):

matrix = pd.read_json('http://router.project-osrm.org/table/v1/driving/13.388860,52.517037;13.397634,52.529407;13.428555,52.523219?annotations=duration,distance')
durations = matrix[['durations']]
distances = matrix[['durations']]

In [219]:
distances

[0,
 275607.2,
 10045.2,
 24120.6,
 297674.7,
 458505.4,
 927374.1,
 170773,
 289196.6,
 178013.8,
 99996.4,
 266280.4,
 34107.3,
 1088619.2,
 55081,
 28189,
 149473.2,
 492965.5,
 210831.3,
 467326.6,
 113778.7,
 137890.1,
 130753.4,
 13650.1,
 36851.1,
 30965,
 24860.2,
 2291795.6,
 572520,
 257859.7]

In [220]:
durations

[0,
 12027.4,
 657.7,
 1294.5,
 12847.6,
 19750.6,
 40578.5,
 8371.9,
 12702,
 8087.8,
 4869.3,
 11525.9,
 1888.9,
 48654.2,
 2842.5,
 2029.5,
 18254.7,
 33008.5,
 8838.4,
 31530.2,
 16461.6,
 17937.9,
 17542.6,
 1051.2,
 2588.4,
 2052.2,
 2210.2,
 101021.7,
 24759.6,
 11159]

In [218]:
from itertools import tee
#use pairwise function to be used to iterate through two consecutive rows (pairs) in a data frame
def pairwise(iterable):
    a, b = tee(iterable)
    next(b, None)
    return zip(a, b)

#Perform request to use the Google Maps API web service
#API_key = 'xxxxxxx'#enter Google Maps API key
#gmaps = googlemaps.Client(key=API_key)

#empty lists - will be used to store calculated distances and durations
durations = [0]
distances = [0]

mode ='driving'

# ORSM route URI for driving mode
route_uri = 'http://router.project-osrm.org/route/v1/driving/'

# Loop through each row in the data frame using pairwise
for (i1, row1), (i2, row2) in pairwise(cities.iterrows()):
      #Assign latitude and longitude as origin/departure points
      LatOrigin = row1['Latitude']
      LongOrigin = row1['Longitude']
      origin = (LatOrigin,LongOrigin)

      #Assign latitude and longitude from the next row as the destination point
      LatDest = row2['Latitude']   # Save value as lat
      LongDest = row2['Longitude']  # Save value as lat
      destination = (LatDest,LongDest)

      #pass origin and destination variables to Google distance_matrix function# output in meters
      # result = gmaps.distance_matrix(origins, destination, mode=mode)["rows"][0]["elements"][0]["distance"]["value"]

      # Location pair of coordinates
      loc_pair = str(LongOrigin) + ',' + str(LatOrigin) + ';' + str(LongDest) + ',' + str(LatDest)

      #pass the location pairs to the OSRM API, read to json and extract routs and duration
      result = pd.read_json( route_uri + loc_pair,lines=True)
      distance = result['routes'][0][0]['distance']
      duration = result['routes'][0][0]['duration']

      #append result to list
      distances.append(distance)
      durations.append(duration)

#Add column 'Distance' to data frame and assign to list values
cities['Distance'] = distances
cities['Duration'] = durations

In [212]:
cities

,Destination Province,Destination City,Latitude,Longitude,Distance,Duration
0,AB,ACHESON,53.545914,-113.771576,0.0,0.0
1,AB,AIRDRIE,51.285970,-114.010620,275607.2,12027.4
2,AB,BALZAC,51.215611,-114.004558,10045.2,657.7
3,AB,CALGARY,51.046095,-114.065465,24120.6,1294.5
4,AB,EDMONTON,53.535411,-113.507996,297674.7,12847.6
5,AB,GRANDE PRAIRIE,55.171080,-118.794987,458505.4,19750.6
6,AB,LETHBRIDGE,49.694285,-112.851562,927374.1,40578.5
7,AB,MEDICINE HAT,50.043030,-110.679016,170773.0,8371.9
8,AB,OKOTOKS,50.725369,-113.975082,289196.6,12702.0
9,AB,RED DEER COUNTY,52.104022,-113.767473,178013.8,8087.8
